In [22]:
from typing import Dict, Optional
import os

import httpx
from simpleaichat import AIChat

MODEL = 'gpt-4-0125-preview'
MODEL_PARAMS = {'temperature':0.0}

In [19]:
def chat_with_memory():
    assert os.environ['OPENAI_API_KEY']
    ai = AIChat(console=False, model=MODEL, params=MODEL_PARAMS)
    prompts = [
        'my name is henry',
        '1+1=?',
        "what's my name?"
    ]
    for prompt in prompts:
        print(ai(prompt))
        session = ai.get_session()
        print(f'token(s): {session.total_length}')

chat_with_memory()

Hello, Henry! How can I assist you today?
token(s): 33
1 + 1 = 2
token(s): 85
Your name is Henry.
token(s): 155


In [18]:
def save_and_load_messages():
    assert os.environ['OPENAI_API_KEY']

    ai_1 = AIChat(console=False, model=MODEL, params=MODEL_PARAMS)
    prompt = 'my name is henry'
    print(ai_1(prompt))

    messages = ai_1.get_session().messages
    ai_2 = AIChat(console=False, model=MODEL, params=MODEL_PARAMS, messages=messages)

    prompt = "what's my name?"
    print(ai_2(prompt))

    session = ai_2.get_session()
    print(f'token(s): {session.total_length}')

save_and_load_messages()

Hello, Henry! How can I assist you today?
Your name is Henry. How can I help you further?
token(s): 58


In [17]:
def weather_tool(query: str):
    '''Useful to query current weather of a location'''

    # Caveat: "query" is user input, not the location
    ai = AIChat(console=False, model=MODEL, params=MODEL_PARAMS)
    location = ai(f'''
    Extract the location from the query.
    QUERY: `{query}`
    LOCATION:
    ''').lower()
    assert location == 'taipei', 'expect the location to be taipei'

    resp = httpx.get(f'https://wttr.in/{location}?format=3')
    _, condition = resp.text.split(':')

    return {'context':f'Weather of {location} is {condition}.'}

def use_tool():
    assert os.environ['OPENAI_API_KEY']
    ai = AIChat(console=False, model=MODEL, params=MODEL_PARAMS)
    print(ai("what's the weather of Taipei", tools=[weather_tool])['response'])
    session = ai.get_session()
    print(f'token(s): {session.total_length}')

use_tool()

The weather in Taipei is partly cloudy with a temperature of +25°C.
token(s): 136


In [31]:
class Toolset:
    def __init__(self, *, ai: AIChat, session_id: Optional[str]=None):
        self.ai = ai
        self.session = ai.get_session(id=session_id)

    @property
    def tools(self):
        return [self.weather_tool]

    def extract_from_messages(self, *, prompt: str, query: str) -> str:
        messages = '===\n'.join([
            f'''
            <role> {m.role}
            <message> {m.content}
            '''  for m in self.session.messages
        ])
        prompt = f'''
        {prompt}
        CONTEXT:
        ===
        {messages}
        ===
        MENTIONED:
        '''
        ai = AIChat(console=False, model=MODEL, params=MODEL_PARAMS)
        return ai(prompt)

    def weather_tool(self, query: str) -> Dict[str, str]:
        '''Useful to query current weather of a location'''
        location = self.extract_from_messages(
            prompt="what's the actual location that the user mentioned in the query?",
            query=query
        ).lower()
        assert location == 'taipei', 'expect the location to be taipei'
        resp = httpx.get(f'https://wttr.in/{location}?format=3')
        _, condition = resp.text.split(':')
        return {'context':f'Weather of {location} is {condition}.'}

def memory_and_tool():
    assert os.environ['OPENAI_API_KEY']
    
    ai = AIChat(console=False, model=MODEL, params=MODEL_PARAMS)
    toolset = Toolset(ai=ai)

    prompts = [
        'my name is henry and i am live in taipei',
        "what's the weather of the place I live?"
    ]
    for prompt in prompts:
        print(ai(prompt, tools=toolset.tools)['response'])

    session = ai.get_session()
    print(f'token(s): {session.total_length}')

memory_and_tool()

Hello Henry! It's great to meet you. How can I assist you today?
The weather in Taipei is currently partly cloudy with a temperature of +25°C. How can I further assist you with this information, Henry?
token(s): 350
